In [8]:
from collections import deque
import pydot
import argparse
import os

# set it to bin folder of graphvviz
os.environ["PATH"] += os.pathsep 

options = [(1, 0), (0, 1), (1, 1), (0, 2), (2,0)]
Parent = dict()
graph= pydot.Dot(graph_type='graph', strict=False, bgcolor="#fff3af",
                 label= "fig:Missionaries and Cannibal State Space Tree",
                 fontcolor="red", fontsize= "24", overlap= "true")

i=0

arg= argparse.ArgumentParser()
arg.add_argument("-d", "--depth", required=False,
                 help="Maximun depth upto which you want to generate Space State Tree")

args = vars(arg.parse_args())
max_depth= int(args.get("depth", 20))

def is_valid_move(number_missionaries, number_cannibals):
    """
    Checks if number 
    """
    return (0 <= number_missionaries <=3) and (0 <= number_cannibals <=3)

def write_image(file_name="state_space"):
    try:
        graph.write_png(f"{file_name}_{max_depth}.png")
    except Exception as e:
        print("Error while writting file", e)
    print(f"File {file_name}_{max_depth}.png successfully written.")

def draw_edge(number_missionaries, number_cannibals, side, depth_level, node_num):
    u, v = None, None
    if Parent[(number_missionaries, number_cannibals, side, depth_level, node_num)] is not None:
        u = pydot.Node(str(Parent[(number_missionaries, number_cannibals, side, depth_level, node_num)]),
                       label= str(Parent[(number_missionaries, number_cannibals, side, depth_level, node_num)][:3]))
        graph.add_node(u)
        v = pydot.Node(str((number_missionaries, number_cannibals, side, depth_level, node_num)),
                       label= str((number_missionaries, number_cannibals, side)))
        graph.add_node(v)

        edge = pydot.Edge(str(Parent[(number_missionaries, number_cannibals, side, depth_level, node_num)]),
                          str((number_missionaries, number_cannibals, side, depth_level, node_num) ), dir= 'forward')
        graph.add_edge(edge)
    else:
        #For start node
        v = pydot.Node(str((number_missionaries, number_cannibals, side, depth_level, node_num)),
                       label= str((number_missionaries, number_cannibals, side)))
        graph.add_node(v)
    return u,v
def is_start_state(number_missionries, number_cannibals, side):
    return (number_missionries, number_cannibals, side) == (3, 3, 1)

def is_goal_state(number_missionries, number_cannibals, side):
    return (number_missionries, number_cannibals, side) == (0, 0, 0)

def number_of_cannibals_exceeds(number_missionries, number_cannibals):
    number_missionries_right = 3 - number_missionries
    number_cannibals_right = 3- number_cannibals
    return (number_missionries > 0 and number_cannibals > number_missionries)\
or (number_missionries_right > 0 and number_cannibals_right> number_missionries_right)

def generate():
    global i
    q= deque()
    node_num = 0
    q.append((3, 3, 1, 0, node_num))

    Parent [(3, 3, 1, 0, node_num)] = None

    while q:

        number_missionaries, number_cannibals, side, depth_level, node_num = q.popleft()
        #print ( number_missionaries, number_cannibals  )
        #Draw Edge from u -> v
        #Where u = Parent[v]
        # and v= (number_missionaries, number_cannibals, side, depth_level)
        u, v = draw_edge(number_missionaries, number_cannibals, side, depth_level, node_num)

        if is_start_state(number_missionaries, number_cannibals, side):
            v.set_style("filled")
            v.set_fillcolor("blue")
            v.set_fontcolor("white")
        elif is_goal_state(number_missionaries, number_cannibals, side):
            v.set_style("filled")
            v.set_fillcolor("green")
            continue    
            # return True
        elif number_of_cannibals_exceeds(number_missionaries, number_cannibals):
            v.set_style("filled")
            v.set_fillcolor("red")
            continue    
        else:
            v.set_style("filled")
            v.set_fillcolor("orange")

        if depth_level == max_depth:
            return True
        
        op = -1 if side ==1 else 1

        can_be_expanded = False

        # i = node_num
        for x, y in options:
            next_m, next_c, next_s = number_missionaries + op * x, number_cannibals + op*y, int(not side)

            if Parent[(number_missionaries, number_cannibals, side, depth_level, node_num)] is None or(next_m, next_c, next_s)\
            != Parent[(number_missionaries, number_cannibals, side, depth_level, node_num)][:3]:
                if is_valid_move(next_m, next_c):
                    can_be_expanded = True
                    i += 1
                    q.append((next_m, next_c, next_s, depth_level+1, i))
                    #Keep track of parent
                    Parent[(next_m, next_c, next_s, depth_level+1, i)] =\
                    (number_missionaries, number_cannibals, side, depth_level, node_num)
        if not can_be_expanded:
            v.set_style("filled")
            v.set_fillcolor("gray")
    return False

if __name__ == "__main__":
    if generate():
        write_image()

usage: ipykernel_launcher.py [-h] [-d DEPTH]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/duongdz/.local/share/jupyter/runtime/kernel-v3d9839fee991659cfb798939eda391b2ec9cbbb37.json


SystemExit: 2

In [ ]:
import os
import emoji
import pydot
import random
from collections import deque


# set it to bin folder of graphvviz
os.environ["PATH"] += os.pathsep


# Dictionaries to backtrack solotion nodes
# Parent stores parent of (m, c, s)
# Move stores (x, y, side) i.e number of missionaries,
#canniabls to be moved from left to right or right to left for particular state
# node_list stores pydot.Node object fpr particular state (m, c, s) so that we can color the solotion nodes
Parent, Move, node_list = dict(), dict(), dict()


class Solution():
   def __init__(self):
       # Start state (3M, 3C, Left)
       # Goal state (0M, 0C, Right)
       # Each state gives the number of missionaries and cannibals on the left side


       self.start_state = (3, 3, 1)
       self.goal_state = (0, 0, 0)
       self.options = [(1, 0), (0, 1), (1, 1), (0, 2), (2,0)]


       self.boat_side = ["right", "left"]


       graph= pydot.Dot(graph_type='graph', strict=False, bgcolor="#fff3af",
                label= "fig:Missionaries and Cannibal State Space Tree", fontcolor="red", fontsize= "24")
       self.visited = {}
       self.solved = False
   def is_valid_move(self, number_missionaries, number_cannibals):
       """
       Check if number constaints are statisfied
       """
       return (0 <= number_missionaries <=3) and (0 <= number_cannibals <= 3)
  
   def is_goal_state(self, number_missionaries, number_cannibals, side):
       return (number_missionaries, number_cannibals, side) == self.goal_state
  
   def is_start_state(self, number_missionaries, number_cannibals, side):
       return (number_missionaries, number_cannibals, side) == self.start_state
  
   def number_of_cannibals_exceeds(self, number_missionaries, number_cannibals):
       number_missionaries_right = 3 - number_missionaries
       number_cannibals_right = 3 - number_cannibals
       return (number_missionaries > 0 and number_cannibals > number_missionaries) \
       or (number_missionaries_right > 0 and number_cannibals_right > number_missionaries_right)      
   def write_image(self, file_name="state_space.png"):
       try:
           self.graph.write_png(file_name)
       except Exception as e:
           print("Error writing file ", e)
       print(f"File  {file_name} written successfully")


   def solve(self, solve_method="dfs"):
       self.visited = dict()
       Parent[self.start_state] = None
       Move[self.start_state] = None
       node_list[self.start_state] = None


       return self.dfs(*self.start_state, 0) if solve_method == "dfs" else self.bfs()
      
   def draw_legend(self):
       """
       Utility method to draw legend on graph if legend flag is on
       """
       graphlegend = pydot.Cluster(graph_name="Legend", label="Legend", frontsize="20", color="gold",
                                   frontclolor="blue", style="filled", fillcolor="#f4f4f4")
      
       node1 = pydot.Node("1", styles="filled", fillcolor="blue", label="Start Node", frontcolor="white", width="2", fixedsize="true")
       graphlegend.add_node(node1)


       node2 = pydot.Node("2", styles="filled", fillcolor="red", label="Killed Node", frontcolor="black", width="2", fixedsize="true")
       graphlegend.add_node(node2)


       node3 = pydot.Node("2", styles="filled", fillcolor="yellow", label="Solution Node", width="2", fixedsize="true")
       graphlegend.add_node(node3)


       node4 = pydot.Node("4", styles="filled", fillcolor="gray", label="Can't be expanded", width="2", fixedsize="true")
       graphlegend.add_node(node4)


       node5 = pydot.Node("5", syles="filled", fillcolor="green", label="Goal Node", width="2", fixedsize="true")
       graphlegend.add_node(node5)


       node7 = pydot.Node("7", styles="filled", fillcolor="gold", label="Node with child", width="2", fixedsize="true")
       graphlegend.add_node(node7)


       description = "Each node (m, c, s) represents a \nstate where 'm' is the number of\n missionaries,\ 'c' is the cannibals \
           and \n's' is the side of the boat\n"
       " Where 'l' represents the left \nside and '0' the right side \n\nOur objective is to reach goal state (0, 0, 0)\
       \nfrom start state (3, 3, 1) by some \noperations = [(1, 0), (0, 1), (2, 0), (0, 2), (1, 1)]\n"\
               "Each tuple (x, y) inside operators \nrepresents the number of missionaries and\
       \ncannibals to be moved from left to right \nif s == 1 and vice versa."


       node6 = pydot.Node("6", styles="filled", fillcolor="gold", label=description, shape="plaintext", frontsize=20, frontcolor="red")
       graphlegend.add_node(node6)
      
       self.graph.add_node(graphlegend)


       self.graph.add_edge(pydot.Edge(node1, node2, style="invis"))
       self.graph.add_edge(pydot.Edge(node2, node3, style="invis"))
       self.graph.add_edge(pydot.Edge(node3, node4, style="invis"))
       self.graph.add_edge(pydot.Edge(node4, node5, style="invis"))
       self.graph.add_edge(pydot.Edge(node5, node7, style="invis"))
       self.graph.add_edge(pydot.Edge(node7, node6, style="invis"))


   def draw(self, *, number_missionaries_left, number_cannibals_left, number_missionaries_right, number_cannibals_right):
       """
               raw state on console using emojis
       """
       left_m = emoji.emojize(f"old_man: " * number_missionaries_left)
       left_c = emoji.emojize(f"ogre: " * number_cannibals_left)
       right_m = emoji.emojize(f":old_man: " * number_missionaries_right)
       right_c = emoji.emojize(f":ogre: " * number_cannibals_right)


       print('{}{}{}{}{}'.format(left_m, left_c, " " * (14 - len(left_m) - len(left_c)),\
                                                               "_" * 40, " " * (12 - len(right_m) - len(right_c)) + right_m, right_c))
       print("")


   def show_solution(self):
       # Recursively start from Goal State
       # And find parent until start state is reached


       state = self.goal_state
       path, steps, nodes = [], [], []


       while state is not None:
           path.append(state)
           steps.append(Move[state])
           nodes.append(node_list[state])


           state = Parent[state]


       steps, nodes = steps[::-1], nodes[::-1]


       number_missionaries_left, number_cannibals_left, side = 3, 3
       number_missionaries_right, number_cannibals_right = 0, 0


       print("*" * 60)
       self.draw(number_missionaries_left=number_missionaries_left, number_cannibals_left=number_cannibals_left,
                   number_missionaries_right=number_missionaries_right, number_cannibals_right=number_cannibals_right)
      
       for i, ((number_missionaries, number_cannibals, side), node) in enumerate(zip(steps[1:])):
          
           if node.get_label() != str(self.start_state):
               node.set_style("filled")
               node.set_fillcolor("yellow")


           print(f"Step {i+1}: Move {number_missionaries} missionaries and {number_cannibals} cannibals \
                   cannibals from {self.board_side[side]} to {self.board_side[int(not side)]}")
          
           op = -1 if side == 1 else 1


           number_missionaries_left = number_missionaries_left + op * number_missionaries
           number_cannibals_left = number_cannibals_left + op * number_cannibals


           number_missionaries_right = number_missionaries_right - op * number_missionaries
           number_cannibals_right = number_cannibals_right - op * number_cannibals


           self.draw(number_missionaries_left=number_missionaries_left, number_cannibals_left=number_cannibals_left,
                       number_missionaries_right=number_missionaries_right, number_cannibals_right=number_cannibals_right)
          
       print("Congratulations!!! You have solve the problem")
       print("*" * 60)


   def draw_edge(self, number_missionaries, number_cannibals, side, depth_level):
       u, v = None, None
       if Parent[(number_missionaries, number_cannibals, side)] is not None:
           u = pydot.Node(str(Parent[(number_missionaries, number_cannibals, side)] + (depth_level - 1,)),
                               label=str(Parent[((number_missionaries, number_cannibals, side))]))
           self.graph.add_node(u)


           v = pydot.Node(str((number_missionaries, number_cannibals, side) + (depth_level)),
                               label=str((number_missionaries, number_cannibals, side)))
           self.graph.add_node(v)


           edge = pydot.Edge(str(Parent[(number_missionaries, number_cannibals, side)] + (depth_level - 1,)),
                               str((number_missionaries, number_cannibals, side)), dir="forward")
           self.graph.add_edge(edge)
       else:
           v = pydot.Node(str((number_missionaries, number_cannibals, side, depth_level)),
                               label=str((number_missionaries, number_cannibals, side)))
           self.graph.add_node(v)
           return u, v
def bfs(self):
    q = deque()
    q.append(self.start_state + (0,))
    self.visited[self.start_state] = True
    
    while q:
        number_missionaries, number_cannibals, side, depth_level = q.popleft()
        
        # Draw Edge from u -> v
        # Where u = Parent[v]
        # and v = (number_missionaries, number_cannibals, side, depth_level)
        u, v = self.draw_edge(number_missionaries, number_cannibals, side, depth_level)
        
        if self.is_start_state(number_missionaries, number_cannibals, side):
            v.set_style("filled")
            v.set_fillcolor("blue")
            v.set_fontcolor("white")
        elif self.is_goal_state(number_missionaries, number_cannibals, side):
            v.set_style("filled")
            v.set_fillcolor("green")
            return True
        elif self.number_of_cannibals_exceeds(number_missionaries, number_cannibals):
            v.set_style("filled")
            v.set_fillcolor("red")
            return False
        else:
            v.set_style("filled")
            v.set_fillcolor("orange")

        op = -1 if side == 1 else 1
        can_be_expanded = False

        for x, y in self.options:
            next_m = number_missionaries + op * x
            next_c = number_cannibals + op * y
            next_s = int(not side)

            if (next_m, next_c, next_s) not in self.visited:
                if self.is_valid_move(next_m, next_c):
                    can_be_expanded = True
                    self.visited[(next_m, next_c, next_s)] = True
                    q.append((next_m, next_c, next_s, depth_level + 1))

                    # Keep track of parent and corresponding move
                    Parent[(next_m, next_c, next_s)] = (number_missionaries, number_cannibals, side)
                    Move[(next_m, next_c, next_s)] = (x, y, side)
                    node_list[(next_m, next_c, next_s)] = v

        if not can_be_expanded:
            v.set_style("filled")
            v.set_fillcolor("gray")

    return False


def dfs(self, number_missionaries, number_cannnibals, side, depth_level):
    self.visited[(number_missionaries, number_cannnibals, side)] = True

    # Draw Edge (from u -> v)
    # Where u = Parent[v]
    u, v = self.draw_edge(number_missionaries, number_cannnibals, side, depth_level)

    if self.is_start_state(number_missionaries, number_cannnibals, side):
        v.set_style("filled")
        v.set_fillcolor("blue")
    elif self.is_goal_state(number_missionaries, number_cannnibals, side):
        v.set_style("filled")
        v.set_fillcolor("green")
        return True
    elif self.number_of_cannibals_exceeds(number_missionaries, number_cannnibals):
        v.set_style("filled")
        v.set_fillcolor("red")
        return False
    else:
        v.set_style("filled")
        v.set_fillcolor("orange")
    solution_found = False                              
    operation = -1 if side == 1 else 1
    can_be_expanded = False
    for x, y in self.options:
        next_m, next_c, next_s = number_missionaries + operation * x, number_cannnibals + operation * y, int(not side)
        
        if (next_m, next_c, next_s) not in self.visited:
            if self.is_valid_move(next_m, next_c):
                can_be_expanded = True
                # Keep track of Parent state and corresponding move
                Parent[(next_m, next_c, next_s)] = (number_missionaries, number_cannnibals, side)
                Move[(next_m, next_c, next_s)] = v
                
                solution_found = (solution_found or self.dfs(next_m, next_c, next_s, depth_level + 1))
                
                if solution_found:
                    return True
                
    if not can_be_expanded:
        v.set.style("filled")
        v.set.fillcolor("gray")
        
    self.solved = solution_found
    return solution_found

In [ ]:
from solve import Solution
import argparse
import itertools

arg = argparse.ArgumentParser()
arg.add_argument("-m", "--method", required=False, help="Specify which method to use")
arg.add_argument("-l", "--legend", required=False, help="Specify if you want to display legend on graph")

args = vars(arg.parse_args())

solve_method = args.get("method", "bfs")
legend_flag = args.get("legend", False)

def main():
    s = Solution()

    if s.solve(solve_method):
        # Display Solution on console
        s.show_solution()

        output_file_name = f"{solve_method}"
        # Draw legend if legend_flag is set
        if legend_flag:
            if legend_flag[0].upper() == 'T':
                output_file_name += "_legend.png"
                s.draw_legend()
            else:
                output_file_name += ".png"
        else:
            output_file_name += ".png"

        # Write State space tree
        s.write_image(output_file_name)
    else:
        raise Exception("No solution found")

if __name__ == "__main__":
    main()